In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def GetData(df):
    cols = df.columns[:-1]
    X = df[cols].to_numpy()
    Y=df['signal'].to_numpy().reshape((len(df),1))
    return X,Y

xtrain, ytrain = GetData(pd.read_csv("train_bonus.csv"))
xtest, ytest   = GetData(pd.read_csv("test_bonus.csv"))

print(xtrain.shape, ytrain.shape)

In [ ]:
import tensorflow.keras
import tensorflow.keras.layers as layers
from tensorflow.keras import Sequential
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)

def SimpleScore(pred, ytest):
    if (pred.shape != ytest.shape): 
        pred=pred.reshape(ytest.shape) # important! prevents broadcasting
    diff = np.abs(pred - np.clip(ytest,0,1))
    return np.mean(np.less(diff,0.5))  

In [ ]:
# this is a start point of your analysis, ... but not a good one
model = Sequential([
    layers.Dense(100, activation="sigmoid"),
    layers.Dense(100, activation="sigmoid"),
    layers.Dense(1, activation="sigmoid")
    ])

model.compile(optimizer="adam", loss='binary_crossentropy') 
model.fit(xtrain, ytrain, epochs=30, verbose=1, batch_size=16, validation_data=(xtest, ytest), callbacks=[early_stop])

print("test score", SimpleScore(model.predict(xtest),ytest), "train score", SimpleScore(model.predict(xtrain),ytrain))